ETAP 1: Wczytanie danych, NLP preprocessing, Wektoryzacja i Feature Importance.

        Wydrukowanie pierwszych wyników przed selekcją cech.

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

spam_dataset = pd.read_csv('spam.csv', encoding="ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'], skiprows=1)

# Konwersja etykiet 'ham' i 'spam' na wartości binarne
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])

# Usuwanie interpunkcji
import string
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(remove_punctuation)

# Tokenizacja tekstu
nltk.download('punkt')
def tokenize(text):
    return nltk.word_tokenize(text.lower())

spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(tokenize)

# Usuwanie stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words("english"))

def remove_stopwords(text):
    return [word for word in text if word not in stopwords_set]

spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(remove_stopwords)

# Lematyzacja
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatizing(text):
    return [lemmatizer.lemmatize(word) for word in text]

spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lemmatizing)

# Wektoryzacja (CountVectorizer i TfidfVectorizer)**
count_vect = CountVectorizer(min_df=0.01, max_df=0.5)
tfidf_vect = TfidfVectorizer(min_df=0.01, max_df=0.5)

# Konwersja listy tokenów na stringi
text_data = spam_dataset['Lemmatized_Text'].apply(lambda x: ' '.join(x))

# Wektoryzacja
X_count = count_vect.fit_transform(text_data)
X_tfidf = tfidf_vect.fit_transform(text_data)

#  Wybór jednej metody wektoryzacji** (możesz testować obie)
X = X_tfidf  # Możesz zamienić na X_count
y = spam_dataset['Spam']

# Podział zbioru na treningowy i testowy (stratyfikacja)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Nauka modelu Random Forest na pełnym zestawie cech
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Feature Importance - istotność cech
importances = rf.feature_importances_

#  Wydruk wyników przed selekcją cech
print(f"Dokładność modelu na pełnym zestawie cech: {rf.score(X_test, y_test):.4f}")
print("Top 20 cech (słów):", list(count_vect.vocabulary_.keys())[:20])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mnkku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mnkku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mnkku\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Dokładność modelu na pełnym zestawie cech: 0.9677
Top 20 cech (słów): ['go', 'great', 'got', 'wat', 'ok', 'free', 'win', 'text', 'txt', 'say', 'already', 'dont', 'think', 'life', 'around', 'hey', 'week', 'back', 'like', 'still']


Model na pełnym zbiorze cech osiągnął 96.77% dokładności, sprawdzimy, czy można go ulepszyć poprzez usunięcie mniej istotnych cech i znalezienie optymalnych parametrów za pomocą GridSearchCV.

 ETAP 2 -  Selekcja cech i ocena modelu przed GridSearch.
 
Usuwam mniej ważne cechy i sprawdzam, jak wpływa to na dokładność modelu.

In [5]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Selekcja cech o ważności > 0.001
important_features = np.where(importances > 0.001)[0]
X_train_selected = X_train[:, important_features]
X_test_selected = X_test[:, important_features]

# Trening nowego modelu na wybranych cechach
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Ocena modelu na zbiorze testowym po selekcji cech
accuracy_selected = rf_selected.score(X_test_selected, y_test)

# Wydruk wyników przed GridSearch
print(f"Liczba cech przed selekcją: {X_train.shape[1]}")
print(f"Liczba cech po selekcji: {X_train_selected.shape[1]}")
print(f"Dokładność modelu po selekcji cech: {accuracy_selected:.4f}")


Liczba cech przed selekcją: 129
Liczba cech po selekcji: 95
Dokładność modelu po selekcji cech: 0.9650


Wnioski:

     >Usunięcie mniej istotnych cech zmniejszyło liczbę wymiarów z 129 do 95, co sprawia, że model jest szybszy i 
      bardziej efektywny.
     > Dokładność modelu minimalnie spadła (z 96.77% do 96.50%), ale model jest teraz prostszy i bardziej zrozumiały.  
     > Możemy teraz przeprowadzić GridSearch, aby sprawdzić, czy dostrojenie hiperparametrów poprawi wynik.

 ETAP 3 -  GridSearch (Dobór najlepszych hiperparametrów).
 
Wykonuje optymalizację hiperparametrów modelu na wybranych cechach.

In [7]:
from sklearn.model_selection import GridSearchCV

# Grid Search - optymalizacja hiperparametrów
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10]
}

#  n_jobs wynosi 1, aby uniknąć problemów z pamięcią
grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                           param_grid, cv=5, n_jobs=1)

grid_search.fit(X_train_selected, y_train)

# Ocena modelu na zbiorze testowym
best_model = grid_search.best_estimator_
accuracy_optimized = best_model.score(X_test_selected, y_test)

#  Wydruk wyników po GridSearch
print(f"Najlepsze hiperparametry: {grid_search.best_params_}")
print(f"Dokładność modelu po optymalizacji hiperparametrów: {accuracy_optimized:.4f}")


Najlepsze hiperparametry: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Dokładność modelu po optymalizacji hiperparametrów: 0.9623


Wyniki i Wnioski : 

- **Liczba cech przed selekcją**: 129
- **Liczba cech po selekcji**: 95
- **Dokładność modelu po selekcji cech**: 96.50%
- **Najlepsze hiperparametry** po GridSearch:
  - `max_depth`: None
  - `min_samples_split`: 2
  - `n_estimators`: 50
- **Dokładność modelu po optymalizacji hiperparametrów**:

1. **Selekcja cech pozwoliła na zmniejszenie liczby atrybutów o 26%** przy zachowaniu wysokiej dokładności.  
2. **Optymalizacja hiperparametrów nieco zmniejszyła dokładność, ale poprawiła stabilność modelu**.  
3. **Model skutecznie klasyfikuje wiadomości e-mail jako SPAM/NON-SPAM i jest gotowy do użycia na nowych danych* 96.23%